In [ ]:
# Import Lib
import os
import cv2
import numpy as np
#from patchify import patchify
from PIL import Image
#from tensorflow.keras.metrics import MeanIoU
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.utils import get_source_inputs
import glob
import imageio.v2 as imageio
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
#from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
import tifffile
import rasterio

root_directory = r'/home/jacks.local/sshinjini/research/'

import rasterio

patch_size = 128

X_train = []
y_train = []
X_test = []
y_test = []

# Directory structure: root_directory/train/image
train_image_directory = os.path.join(root_directory, "train", "image")
train_mask_directory = os.path.join(root_directory, "train", "mask")
test_image_directory = os.path.join(root_directory, "test", "image")
test_mask_directory = os.path.join(root_directory, "test", "mask")

# Function to read and process image patches
def read_and_process_images(directory, is_mask=False):
    data = []
    for path, subdirs, files in os.walk(directory):
        for image_name in files:
            if image_name.endswith(".tif"):
                image_path = os.path.join(path, image_name)
                if is_mask:
                    image = cv2.imread(image_path, 0)  # Read mask patches as grayscale (single-channel)
                    # Add band dimension to the mask
                    image = image[:, :, np.newaxis]
                else:
                    # Read TIFF using rasterio
                    with rasterio.open(image_path) as src:
                        image = src.read()  # Reads all bands
                        nodata = src.nodata  # Get NoData value if present
                        if nodata is not None:
                            image[image == nodata] = 0  # Replace NoData values

                        image = np.moveaxis(image, 0, -1)  # Convert from (Bands, H, W) to (H, W, Bands)
                data.append(image)
    return data

# Read and process train image patches and masks
X_train = read_and_process_images(train_image_directory)
y_train = read_and_process_images(train_mask_directory, is_mask=True)

# Read and process test image patches and masks
X_test = read_and_process_images(test_image_directory)
y_test = read_and_process_images(test_mask_directory, is_mask=True)

# Optionally, you can convert the lists to NumPy arrays for further processing
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
print(np.unique(y_train))
print(np.unique(y_test))
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=2)
print(y_train.shape)
y_test = to_categorical(y_test, num_classes=2)
print(y_test.shape)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print (X_train[0,0,0,:])
print (X_train[50,30,30,:])
print (X_test [0,0,0,:])

x_train_copy = X_train
X_test_copy = X_test
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_shape = x_train_copy.shape  # Store the original shape
X_train = x_train_copy.reshape(-1, X_train.shape[-1])  # Reshape to 2D array
print(X_train.shape) 
# Fit and transform X_train
mean_train = X_train.mean(axis=0)
std_train = X_train.std (axis=0)
X_train = (X_train-mean_train[np.newaxis,:])/std_train[np.newaxis,:]
print (mean_train)
print (x_train_copy[50,30,30,:])

# X_train = scaler.fit_transform(X_train)

# Reshape X_train back to its original shape
X_train = X_train.reshape(X_train_shape)
print (X_train[0,0,0,:])
print (X_train[50,30,30,:])

# Similarly, transform X_test
X_test_shape = X_test_copy.shape
X_test = X_test.reshape(-1, X_test_copy.shape[-1])
# X_test = scaler.transform(X_test)
X_test = (X_test-mean_train[np.newaxis,:])/std_train[np.newaxis,:]
X_test = X_test.reshape(X_test_shape)
print (x_train_copy[50,30,30,:])
print (X_test[50,30,30,:])
y_train_copy = y_train 
y_test_copy = y_test 
print (y_train.shape)
print(np.unique(y_train))
y_train_pp = (y_train[:,:,:,0]==1).sum()
print (y_train_pp)
y_train_pp = (y_train[:,:,:,1]==1).sum()
print (y_train_pp)
#y_train_pp = (y_train[:,:,:,2]==1).sum()
#print (y_train_pp)
#y_train_pp = (y_train[:,:,:,3]==1).sum()
#print (y_train_pp)
#print (58839+70072+128914+1102047)
#print (1328*32*32)
print (y_train)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

IMG_HEIGHT = X_train.shape[1]
IMG_WIDTH  = X_train.shape[2]
IMG_CHANNELS = X_train.shape[3]

In [ ]:
import tensorflow as tf 
import keras.backend as K
from keras.losses import categorical_crossentropy
from typing import Callable, Union
class_weights = {0: 4.95, 1: 0.56}
weights = [class_weights[i] for i in range(len(class_weights))]

@tf.keras.utils.register_keras_serializable()
def multiclass_weighted_cross_entropy(class_weights = weights, is_logits = False, num_cls=2):
    def loss(y_true, y_pred):
        y_pred_soft = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        # Calculate weighted cross entropy loss
        wce_loss = K.mean(-y_true * K.log(y_pred_soft) * class_weights, axis=-1)

        return wce_loss

    return loss

total_loss = multiclass_weighted_cross_entropy()
print(total_loss)

In [ ]:
import simple_multi_unet_model_attentiongate
from simple_multi_unet_model_attentiongate import multi_unet_model

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

metrics=['accuracy', jacard_coef]

def attention_gate(x, g, inter_shape):
    """
    Attention Gate to refine feature maps by focusing on relevant regions
    x: Input feature map
    g: Skip connection from encoder
    inter_shape: Intermediate shape for convolution
    """
    # Apply 1x1 convolution to reduce dimensions
    theta_x = Conv2D(inter_shape, (1, 1), strides=(1, 1), padding='same')(x)
    phi_g = Conv2D(inter_shape, (1, 1), strides=(1, 1), padding='same')(g)
    
    # Add the two feature maps
    add_xg = Add()([theta_x, phi_g])
    act_xg = Activation('relu')(add_xg)
    
    # Apply convolution to learn spatial attention
    psi = Conv2D(1, (1, 1), padding='same', activation='sigmoid')(act_xg)
    
    # Apply attention to input feature map
    out = Multiply()([x, psi])
    
    return out

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras import backend as K
from tensorflow.keras import regularizers
from keras.layers import Add, Activation, Multiply

n_classes = 2
if True: 
    inputs = X_train[:,:,:,:]
    # inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(s)
    c1 = Dropout(0.2)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    p1b = BatchNormalization()(p1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(p1b)
    c2 = Dropout(0.2)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    p2b = BatchNormalization()(p2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(p2b)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    p3b = BatchNormalization()(p3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(p3b)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
    p4b = BatchNormalization()(p4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(p4b)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(c5)
    
    # Expansive path with Attention Gates
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = attention_gate(u6, c4, 64)  # Apply attention gate
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(c6)
    c6b = BatchNormalization()(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6b)
    u7 = attention_gate(u7, c3, 32)  # Apply attention gate
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(c7)
    c7b = BatchNormalization()(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7b)
    u8 = attention_gate(u8, c2, 16)  # Apply attention gate
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(u8)
    c8 = Dropout(0.2)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(c8)
    c8b = BatchNormalization()(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8b)
    u9 = attention_gate(u9, c1, 8)  # Apply attention gate
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(u9)
    c9 = Dropout(0.2)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', kernel_regularizer=regularizers.l2(0.01))(c9)
     
    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)
    
    print (outputs.shape )

In [ ]:
n_classes = 2

def get_model():
    return multi_unet_model(n_classes=n_classes, IMG_HEIGHT=IMG_HEIGHT, IMG_WIDTH=IMG_WIDTH, IMG_CHANNELS=IMG_CHANNELS)
model = get_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=total_loss, metrics=metrics)
print(model.predict (X_test[:,:,:,:]))
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)
model.summary()
#print (y_train[0,0,:,:])

In [ ]:
model.input_shape
model.output_shape
history1 = model.fit(X_train, y_train, 
                    batch_size = 64, 
                    verbose=1, 
                    epochs=40, 
                    validation_data=(X_test, y_test), 
                    shuffle=True
                    )

In [ ]:
model.save('/home/jacks.local/sshinjini/research/v1_attentiongate.keras')

In [ ]:
from tensorflow.keras.models import load_model

model = load_model(
    '/home/jacks.local/sshinjini/research/v1_attentiongate.keras',
    custom_objects={
        'loss': total_loss,
        'jacard_coef': jacard_coef
    }
)

In [ ]:
y_pred=model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=3)
y_test_argmax=np.argmax(y_test, axis=3)
y_train_argmax=np.argmax(y_train, axis=3)

In [ ]:
print("y_train_argmax shape:", y_train_argmax.shape)
print("y_pred_argmax shape:", y_pred_argmax.shape)
from keras.metrics import MeanIoU
n_classes = 2
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test_argmax, y_pred_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
y_pred=model.predict(X_train)
y_pred_argmax=np.argmax(y_pred, axis=3)
y_train_argmax=np.argmax(y_train, axis=3)
cm_training = confusion_matrix(y_train_argmax.ravel(), y_pred_argmax.ravel())
num_classes = 2
class_labels = ["Slum Area", "Non-Slum Area"]
user_accuracies = []
producer_accuracies = []

for class_idx in range(num_classes):
    TP = cm_training[class_idx, class_idx]
    FN = sum(cm_training[class_idx, :]) - TP
    FP = sum(cm_training[:, class_idx]) - TP

    user_accuracy = (TP / (TP + FN)) * 100
    producer_accuracy = (TP / (TP + FP)) * 100

    user_accuracies.append(user_accuracy)
    producer_accuracies.append(producer_accuracy)

# user_accuracies now contains the user accuracy for each class
# producer_accuracies now contains the producer accuracy for each class

print(cm_training)  # Print the confusion matrix
print("\nClass-wise User Accuracies:")
for i, label in enumerate(class_labels):
    print(f"{label}: {user_accuracies[i]:.2f}%")
print("\nClass-wise Producer Accuracies:")
for i, label in enumerate(class_labels):
    print(f"{label}: {producer_accuracies[i]:.2f}%")

from sklearn.metrics import confusion_matrix,classification_report
y_pred=model.predict(X_test)
y_pred_argmax=np.argmax(y_pred, axis=3)
y_test_argmax=np.argmax(y_test, axis=3)
cm_testing = confusion_matrix(y_test_argmax.ravel(), y_pred_argmax.ravel())
num_classes = 2
class_labels = ["Slum Area", "Non-Slum Area"]
user_accuracies = []
producer_accuracies = []

for class_idx in range(num_classes):
    TP = cm_testing[class_idx, class_idx]
    FN = sum(cm_testing[class_idx, :]) - TP
    FP = sum(cm_testing[:, class_idx]) - TP

    user_accuracy = (TP / (TP + FN)) * 100
    producer_accuracy = (TP / (TP + FP)) * 100

    user_accuracies.append(user_accuracy)
    producer_accuracies.append(producer_accuracy)

# user_accuracies now contains the user accuracy for each class
# producer_accuracies now contains the producer accuracy for each class

print(cm_testing)  # Print the confusion matrix
print("\nClass-wise User Accuracies:")
for i, label in enumerate(class_labels):
    print(f"{label}: {user_accuracies[i]:.2f}%")
print("\nClass-wise Producer Accuracies:")
for i, label in enumerate(class_labels):
    print(f"{label}: {producer_accuracies[i]:.2f}%")

In [ ]:
#f1 score: 128*128 patches (Unet Attention Gate)- Maxar

import numpy as np

def calculate_classwise_f1(conf_matrix, class_names):
    f1_scores = {}
    all_f1s = []
    
    for i, class_name in enumerate(class_names):
        TP = conf_matrix[i, i]
        FP = conf_matrix[:, i].sum() - TP
        FN = conf_matrix[i, :].sum() - TP

        precision = TP / (TP + FP) if (TP + FP) != 0 else 0
        recall    = TP / (TP + FN) if (TP + FN) != 0 else 0
        f1        = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0

        f1_scores[class_name] = f1
        all_f1s.append(f1)

    overall_f1 = sum(all_f1s) / len(all_f1s) if all_f1s else 0
    return f1_scores, overall_f1

# Replace these with your actual confusion matrices
train_conf_matrix = cm_training

test_conf_matrix = cm_testing

class_names = ['Slum area', 'Non-slum area']

# Get class-wise and overall F1 scores
train_f1_scores, train_overall_f1 = calculate_classwise_f1(train_conf_matrix, class_names)
test_f1_scores, test_overall_f1 = calculate_classwise_f1(test_conf_matrix, class_names)

# Print nicely
print("Train F1 Scores:")
for cls, score in train_f1_scores.items():
    print(f"{cls}: {score:.4f}")
print(f"Overall F1 Score: {train_overall_f1:.4f}\n")

print("Test F1 Scores:")
for cls, score in test_f1_scores.items():
    print(f"{cls}: {score:.4f}")
print(f"Overall F1 Score: {test_overall_f1:.4f}")

In [ ]:
#code for images with non-matched labels

import random
test_img_number = random.randint(0, len(X_test) - 1)
test_img = X_test[test_img_number]
ground_truth = y_test_argmax[test_img_number]

# Rescale the RGB values to [0, 1] range
rgb_test_img = (test_img[:, :, :3] - test_img[:, :, :3].min()) / (test_img[:, :, :3].max() - test_img[:, :, :3].min())

test_img_input = np.expand_dims(test_img, 0)
prediction = model.predict(test_img_input)
predicted_img = np.argmax(prediction, axis=3)[0, :, :]

In [ ]:
# Define the colormap for each class
testing_class_cmap = ['red', 'orange']
testing_class_labels = ['Slum area', 'Non-slum Area']

prediction_class_cmap = ['red', 'orange']
prediction_class_labels = ['Slum area', 'Non-slum Area']

plt.figure(figsize=(18, 6))

# Plot the RGB testing image
plt.subplot(231)
plt.title('RGB Testing Image')
plt.imshow(rgb_test_img)
plt.axis('off')

# Plot the testing label with specific colors
plt.subplot(232)
plt.title('Testing Label')
plt.imshow(ground_truth, cmap=plt.cm.colors.ListedColormap(testing_class_cmap))
plt.colorbar(ticks=range(len(testing_class_labels)), format=plt.FuncFormatter(lambda val, loc: testing_class_labels[val]))
plt.axis('off')

# Plot the predicted image with specific colors
plt.subplot(233)
plt.title('Prediction on Test Image')
plt.imshow(predicted_img, cmap=plt.cm.colors.ListedColormap(prediction_class_cmap))
plt.colorbar(ticks=range(len(prediction_class_labels)), format=plt.FuncFormatter(lambda val, loc: prediction_class_labels[val]))
plt.axis('off')

#plt.savefig('v2_5.tif', format='tif')
plt.show()

print("Min value:", np.min(rgb_test_img))
print("Max value:", np.max(rgb_test_img))